In [1]:
import scipy, numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt
%matplotlib inline

def stft(x, fftsize=1024, overlap=4):   
    hop = fftsize / overlap
    w = scipy.hanning(fftsize*2)[:-1] # better reconstruction with this trick +1)[:-1]  
    print x.shape
    return np.array([np.fft.fft(w*interp(x[i:i+fftsize])) for i in range(0, len(x)-fftsize, hop)])

def istft(X, overlap=4):   
    fftsize=(X.shape[1]-1)
    hop = fftsize / overlap
    w = scipy.hanning(fftsize+1)
    print(np.fft.ifft(X[0]).shape)
    print(w.shape)
    x = scipy.zeros(X.shape[0]*hop)
    wsum = scipy.zeros(X.shape[0]*hop) 
    for n,i in enumerate(range(0, len(x)-fftsize, hop)): 
        x[i:i+fftsize] += scipy.real(np.fft.ifft(X[n])) * w   # overlap-add
        wsum[i:i+fftsize] += w ** 2.
    pos = wsum != 0
    x[pos] /= wsum[pos]
    return x

def interp(x):
    x=np.array(x, dtype='float')
    x_d = np.zeros(x.shape[0]*2-1)
    x_m = x[:-1]
    x_p = x[1:]
    x_d[::2] = x
    x_d[1::2] = (x_m+x_p)/2
    return x_d

In [23]:
import scipy, pylab

def stft(x, fs, framesz, hop):
    framesamp = int(framesz*fs)
    hopsamp = int(hop*fs)
    w = scipy.hanning(framesamp)
    X = scipy.array([scipy.fft(w*x[i:i+framesamp]) 
                     for i in range(0, len(x)-framesamp, hopsamp)])
    return X

def istft(X, fs, T, hop):
    x = scipy.zeros(T*fs)
    framesamp = X.shape[1]
    hopsamp = int(hop*fs)
    wsum = scipy.zeros(T*fs)
    w = scipy.hanning(framesamp)
    for n,i in enumerate(range(0, len(x)-framesamp, hopsamp)):
        x[i:i+framesamp] += scipy.real(scipy.ifft(X[n]))
        wsum[i:i+framesamp] += w
    pos = wsum != 0
    x[pos] /= wsum[pos]
    return x

In [34]:
fs, x = wavfile.read('prelude_mono.wav')
T = float(len(x))/fs
framesz = 0.025  # with a frame size of 50 milliseconds
hop = 0.0025
print fs, x.shape

44100 (17437696,)


In [35]:
X = stft(x, fs, framesz, hop)

In [36]:
iX = istft(X, fs, T, hop)
print iX.shape

/home/emmanuel/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:12: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/emmanuel/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:15: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


(17437696,)


In [38]:
ft_x = np.fft.rfft(x)

In [39]:
ift_x = np.fft.irfft(ft_x)

In [41]:
wavfile.write('test3.wav', fs, ift_x)